# Supply Chain Maze

In [48]:
import cv2
import numpy as np
# pip install Pillow
    # open Anaconda Prompt and paste above line (without '#') to install package
from PIL import Image

In [50]:
# function that gives a range on hues given a color
def get_limits(color):
    c = np.uint8([[color]])
    hsvC = cv2.cvtColor(c, cv2.COLOR_BGR2HSV)
    
    lowerLimit = hsvC[0][0][0] - 10, 100, 100
    upperLimit = hsvC[0][0][0] + 10, 255, 255
    # the +/-10 defines the range of hues that fall within the limits (the h in hsv)
    # the range on saturation and value is much bigger because we are only looking for hue
    
    lowerLimit = np.array(lowerLimit, dtype=np.uint8)
    upperLimit = np.array(upperLimit, dtype=np.uint8)

    return lowerLimit, upperLimit

In [52]:
# take from SCM Calibration
#points = [[508, 310], [579, 222], [595, 383], [651, 221], [656, 305], [661, 379], [727, 215], [732, 376], [793, 296]]
points = [[99, 235], [192, 142], [172, 322], [301, 114], [281, 221], [283, 342], [436, 106], [442, 326], [539, 210]]

color = [85, 145, 43]
#color = [90, 144, 31]                # color in BGR colorspace
line_color = [0, 255, 255]            # color used to draw path
line_size = 4                         # line thickness
line_color_check = [0, 0, 255]        # line color used to draw the correct solution
line_depletion = 0.9                  # affects how the color of the solution lines change (>1 makes them get brighter, <1 makes them darker, keep near 1)
detection_size = 500                 # minimum size of detection required to mark color as an object
detection_range = 10                  # determines how close object needs to be to points to see an overlap (higher number -> easier to connect points)
capture = cv2.VideoCapture(0)         # picks camera to use (usually 0 or 1)


################################################################################################################################################
################################################################################################################################################
################################################################################################################################################

test = False
ShortestPath = 'und'

if (line_color == [0, 0, 0]): # breaks if you use black ([0, 0, 0]), so we adjust it
    line_color = [1, 1, 1]
if (line_color_check == [0, 0, 0]): # breaks if you use black ([0, 0, 0]), so we adjust it
    line_color_check = [1, 1, 1]

while True:
    first = False
    count = 0
    prev_node = -1
    current_node = -1
    path_value = 0
    nodes_taken = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0])
    complete = False
    path = 0
                         # node   s   1   2   3   4   5   6   7   e
    adjacency_matrix = np.array([[0,  11, 8,  0,  0,  0,  0,  0,  0 ],   # start
                                 [11, 0,  8,  7,  0,  0,  0,  0,  0 ],   # 1
                                 [8,  8,  0,  0,  2,  7,  0,  0,  0 ],   # 2
                                 [0,  7,  0,  0,  4,  0,  7,  7,  0 ],   # 3
                                 [0,  0,  2,  4,  0,  4,  0,  0,  0 ],   # 4
                                 [0,  0,  7,  0,  4,  0,  0,  7,  0 ],   # 5
                                 [0,  0,  0,  7,  0,  0,  0,  11, 8 ],   # 6
                                 [0,  0,  0,  7,  0,  7,  11, 0,  11],   # 7
                                 [0,  0,  0,  0,  0,  0,  8,  11, 0 ]])  # end
                    # a '1' indicates that two nodes (corresponing to the row and column) are adjacent
                    # the matrix is symmetric, so it doesn't matter if you take [3][4] or [4][3] ; they tell you the same thing
    while True:
        ret, frame = capture.read()
        frame = cv2.flip(frame, 1)
        
        if (first == False): # runs only once
            # get height and width of video
            shape = frame.shape
            height = shape[0]
            width = shape[1]
            cqm = width / 640  # scales detection with video quality (because it depends on number of pixels)
            line_thickness = int(line_size * cqm)
            # create path image to draw on (based on shape of webcam feed)
            path = np.array(Image.new("RGB", (width, height), (0,0,0)))
            # draw dots to mark each node at each node
            for i in points:    # draws markers at every point
                cv2.circle(path, i, 1, (1, 1, 1), int(9*cqm))
                cv2.circle(path, i, 1, line_color, int(6*cqm))
                cv2.circle(path, i, 1, (1, 1, 1), int(3*cqm))
            # key input to decide if we are doing Shortest Path Problem or Traveling Salesman
            while True:
                if ShortestPath == 'und': # skips if Shortest Path is already assigned to True or False
                    cv2.putText(frame, 'press [s] key for Shortest Path Problem', (int(25*cqm), int(height - 60*cqm)), cv2.FONT_HERSHEY_TRIPLEX, 0.5*cqm, (150, 150, 150), int(1*cqm))
                    cv2.putText(frame, 'press [t] key for Traveling Salesman', (int(25*cqm), int(height - 25*cqm)), cv2.FONT_HERSHEY_TRIPLEX, 0.5*cqm, (150, 150, 150), int(1*cqm))
                    key = cv2.waitKey(10) & 0xFF
                    if key == 115:              # 115 is ASCII for [s]
                        ShortestPath = True     # choose Shortest Path Problem
                        break                   # ends loop when [s] is pressed
                    if key == 116:              # 116 is ASCII for [t]
                        ShortestPath = False    # choose Traveling Salesman
                        break                   # ends loop when [t] is pressed
                    if key == 27:               # 27 is ASCII for [ESC]
                        break                   # HOLD [ESC] to end program
                else:
                    break
                cv2.imshow('Supply Chain Maze', frame)
            # set first to True so this block doesn't run again
            first = True
    
        frame_blur = cv2.GaussianBlur(frame, (11, 11), 9) # blurring the image may help get the desired result, but it can be removed
        
        frame_hsv = cv2.cvtColor(frame_blur, cv2.COLOR_BGR2HSV) # convert to HSV
        lowerLimit, upperLimit = get_limits(color) # range of hues that we want the software to detect
        
        mask = cv2.inRange(frame_hsv, lowerLimit, upperLimit) # detects objects in color range
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        bboxes = list()
        for cnt in contours:
            if cv2.contourArea(cnt) > cqm**2*detection_size:  # only continues if size of the object is large enough (removes noise)
                x1, y1, w, h = cv2.boundingRect(cnt)  # finds a bounding box for each object
                c = list([int(x1 + w/2), int(y1 + h/2)])  # centerpoint of bbox
                # check other bboxes to see if we want to combine them into 1 box
                newBox = list([c[0], c[1], x1, y1, w, h, cv2.contourArea(cnt)])
                for i in bboxes[:]:
                    cxi, cyi, x1i, y1i, wi, hi, si = i  # bbox we check the newBox against
                    if np.sqrt((c[0] - cxi)**2 + (c[1] - cyi)**2) < np.sqrt(w**2 + h**2)/3 + np.sqrt(wi**2 + hi**2)/3 + cqm*25: # if centerpoints are close enough (scales with box size)
                        bboxes.remove(i)
                        # reassign bbox boundaries so the new box contains both nearby boxes
                        newBox[2], newBox[3] = min(x1, x1i), min(y1, y1i)  # reassigns x1 and y1 values
                        newBox[4], newBox[5] = max(x1+w, x1i+wi) - newBox[2], max(y1+h, y1i+hi) - newBox[3]  # reassgins w and h values
                        newBox[0], newBox[1] = int(newBox[2] + newBox[4]/2), int(newBox[3] + newBox[5]/2)  # reassigns centerpoint values
                bboxes.append(newBox)
    
        # overlay the path over the webcam feed
        mask2 = cv2.cvtColor(path, cv2.COLOR_BGR2GRAY)
        mask2 = cv2.threshold(mask2, 0, 255, cv2.THRESH_BINARY)[1]
        mask2inv = cv2.bitwise_not(mask2)
        pathfg = cv2.bitwise_and(path, path, mask=mask2) # extract foreground
        framebg = cv2.bitwise_and(frame, frame, mask=mask2inv) # extract background
        frame = cv2.add(framebg, pathfg) # combine foreground and background
        
        # find the largest object
        maxVal = 0
        marker = 0
        for i in bboxes:  
            cx, cy, x1, y1, w, h, s = i
            if s > maxVal:
                maxVal = s
                marker = i
    
        # draw circle around current node
        if current_node != -1:
            cv2.circle(frame, points[current_node], int(20*cqm), (0, 0, 255), int(3*cqm))
            
        # draw bounding box
        if marker != 0:
            cx, cy, x1, y1, w, h, s = marker
            cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 0, 255), 2)
            cv2.line(frame, (cx, cy), (cx, cy), (255, 0, 0), int(15*cqm))
            cv2.line(frame, (cx, cy), (cx, cy), (0, 255, 255), int(10*cqm))
            cv2.line(frame, (cx, cy), (cx, cy), (0, 0, 255), int(5*cqm))
    
            # check if we are close enough to a node
            for index, value in enumerate(points):
                if current_node == -1 and index == 0 and (marker[0]-value[0])**2+(marker[1]-value[1])**2 < cqm*50*detection_range:
                    current_node = 0
                    nodes_taken[index] = 1
                elif (current_node != -1) and (adjacency_matrix[current_node][index] != 0 or test) and (index != prev_node) and ((marker[0]-value[0])**2+(marker[1]-value[1])**2 < cqm*50*detection_range):
                    cv2.line(path, (points[current_node][0], points[current_node][1]), (points[index][0], points[index][1]), line_color, line_thickness)
                    nodes_taken[index] = 1
                    prev_node = current_node
                    current_node = index
                    path_value += adjacency_matrix[prev_node][current_node]
                    if ShortestPath:
                        if index == 8:
                            complete = -1
                    else:
                        if (np.sum(nodes_taken) == 9 and index == 8) or test:
                            complete = -1
        
        #cv2.imshow('blur', frame_blur)
        #cv2.imshow('mask', mask)
        cv2.imshow('Supply Chain Maze', frame)
        
        if complete == True:
            cv2.waitKey(1000)
            break
    
        if complete == -1:
            complete = True
        
        key = cv2.waitKey(10) & 0xFF
        if key == 27:               # 27 is ASCII for [ESC]
            break                   # ends loop when [ESC] is pressed

        key = cv2.waitKey(10) & 0xFF
        if key == 32:               # 32 is ASCII for [spacebar]
            break                   # ends loop when [spacebar] is pressed

    if key == 27:               # 27 is ASCII for [ESC]
        key = 0
        break                   # HOLD [ESC] to end program
    
    ################################################################################################################################################
    ################################################################################################################################################
    ################################################################################################################################################

    if complete == True:
        if ShortestPath: 
            num_nodes = np.sum(nodes_taken) # counts how many nodes were taken in total
            path2 = np.array(Image.new("RGB", (width, height), (0,0,0)))
            
            # draws best path from each node touched to end
            dlt = 2  # number of pixels increase in thickeness for each solution drawn
            lt = int(line_thickness + dlt*(num_nodes-1) + 4)    # line thickness
            lcc = [min(int(i*line_depletion**3), 255) for i in line_color_check]    # line color
            if nodes_taken[7]:
                lt -= dlt      # line thickness decreases for each path drawn, so that different lines are visible
                lcc = [min(int(i*line_depletion), 255) for i in lcc]   # line color changes for each path drawn, so that different lines are visible
                cv2.line(path2, (points[7][0], points[7][1]), (points[8][0], points[8][1]), lcc, lt-3*dlt)
            if nodes_taken[6]:
                lt -= dlt
                lcc = [min(int(i*line_depletion), 255) for i in lcc]
                cv2.line(path2, (points[6][0], points[6][1]), (points[8][0], points[8][1]), lcc, lt)
            if nodes_taken[5]:
                lt -= dlt
                lcc = [min(int(i*line_depletion), 255) for i in lcc]
                cv2.line(path2, (points[5][0], points[5][1]), (points[7][0], points[7][1]), lcc, lt-3*dlt)
                cv2.line(path2, (points[7][0], points[7][1]), (points[8][0], points[8][1]), lcc, lt-3*dlt)
            if nodes_taken[3]:
                lt -= dlt
                lcc = [min(int(i*line_depletion), 255) for i in lcc]
                cv2.line(path2, (points[3][0], points[3][1]), (points[6][0], points[6][1]), lcc, lt)
                cv2.line(path2, (points[6][0], points[6][1]), (points[8][0], points[8][1]), lcc, lt)
            if nodes_taken[4]:
                lt -= dlt
                lcc = [min(int(i*line_depletion), 255) for i in lcc]
                cv2.line(path2, (points[4][0], points[4][1]), (points[3][0], points[3][1]), lcc, lt)
                cv2.line(path2, (points[3][0], points[3][1]), (points[6][0], points[6][1]), lcc, lt)
                cv2.line(path2, (points[6][0], points[6][1]), (points[8][0], points[8][1]), lcc, lt)
            if nodes_taken[2]:
                lt -= dlt
                lcc = [min(int(i*line_depletion), 255) for i in lcc]
                cv2.line(path2, (points[2][0], points[2][1]), (points[4][0], points[4][1]), lcc, lt)
                cv2.line(path2, (points[4][0], points[4][1]), (points[3][0], points[3][1]), lcc, lt)
                cv2.line(path2, (points[3][0], points[3][1]), (points[6][0], points[6][1]), lcc, lt)
                cv2.line(path2, (points[6][0], points[6][1]), (points[8][0], points[8][1]), lcc, lt)
            if nodes_taken[1]:
                lt -= dlt
                lcc = [min(int(i*line_depletion), 255) for i in lcc]
                cv2.line(path2, (points[1][0], points[1][1]), (points[3][0], points[3][1]), lcc, lt)
                cv2.line(path2, (points[3][0], points[3][1]), (points[6][0], points[6][1]), lcc, lt)
                cv2.line(path2, (points[6][0], points[6][1]), (points[8][0], points[8][1]), lcc, lt)
            if nodes_taken[0]:
                lt -= dlt
                lcc = line_color_check   
                cv2.line(path2, (points[0][0], points[0][1]), (points[2][0], points[2][1]), lcc, lt)
                cv2.line(path2, (points[2][0], points[2][1]), (points[4][0], points[4][1]), lcc, lt)
                cv2.line(path2, (points[4][0], points[4][1]), (points[3][0], points[3][1]), lcc, lt)
                cv2.line(path2, (points[3][0], points[3][1]), (points[6][0], points[6][1]), lcc, lt)
                cv2.line(path2, (points[6][0], points[6][1]), (points[8][0], points[8][1]), lcc, lt)
            
            # caluculate and display score
            if test:
                path_value = 8378
            optimal_score = 29
            percent_score = int(100 * optimal_score / path_value)
            border_color = (1, 1, 1)
            cv2.putText(path2, f'Score: {path_value}    {percent_score}%', (int(8*cqm), int(50*cqm)), cv2.FONT_HERSHEY_TRIPLEX, 2*cqm, border_color, int(2*cqm))
            cv2.putText(path2, f'Score: {path_value}    {percent_score}%', (int(12*cqm), int(50*cqm)), cv2.FONT_HERSHEY_TRIPLEX, 2*cqm, border_color, int(2*cqm))
            cv2.putText(path2, f'Score: {path_value}    {percent_score}%', (int(10*cqm), int(48*cqm)), cv2.FONT_HERSHEY_TRIPLEX, 2*cqm, border_color, int(2*cqm))
            cv2.putText(path2, f'Score: {path_value}    {percent_score}%', (int(10*cqm), int(52*cqm)), cv2.FONT_HERSHEY_TRIPLEX, 2*cqm, border_color, int(2*cqm))
            cv2.putText(path2, f'Score: {path_value}    {percent_score}%', (int(10*cqm), int(50*cqm)), cv2.FONT_HERSHEY_TRIPLEX, 2*cqm, (0, 200, 200), int(2*cqm))
    
            #display instrction text
            cv2.putText(path2, 'press [spacebar to reset]', (int(26*cqm), int(height - 25*cqm)), cv2.FONT_HERSHEY_TRIPLEX, 1*cqm, (1, 1, 1), int(1*cqm))
            cv2.putText(path2, 'press [spacebar to reset]', (int(24*cqm), int(height - 25*cqm)), cv2.FONT_HERSHEY_TRIPLEX, 1*cqm, (1, 1, 1), int(1*cqm))
            cv2.putText(path2, 'press [spacebar to reset]', (int(25*cqm), int(height - 26*cqm)), cv2.FONT_HERSHEY_TRIPLEX, 1*cqm, (1, 1, 1), int(1*cqm))
            cv2.putText(path2, 'press [spacebar to reset]', (int(25*cqm), int(height - 24*cqm)), cv2.FONT_HERSHEY_TRIPLEX, 1*cqm, (1, 1, 1), int(1*cqm))
            cv2.putText(path2, 'press [spacebar to reset]', (int(25*cqm), int(height - 25*cqm)), cv2.FONT_HERSHEY_TRIPLEX, 1*cqm, (150, 150, 150), int(1*cqm))
        else:
            num_nodes = np.sum(nodes_taken) # counts how many nodes were taken in total
            path2 = np.array(Image.new("RGB", (width, height), (0,0,0)))
            
            # draws best path from each node touched to end
            lt = int(line_thickness + 6)    # line thickness
            lcc = line_color_check    # line color
            cv2.line(path2, (points[0][0], points[0][1]), (points[1][0], points[1][1]), lcc, lt)
            cv2.line(path2, (points[1][0], points[1][1]), (points[2][0], points[2][1]), lcc, lt)
            cv2.line(path2, (points[2][0], points[2][1]), (points[4][0], points[4][1]), lcc, lt)
            cv2.line(path2, (points[4][0], points[4][1]), (points[5][0], points[5][1]), lcc, lt)
            cv2.line(path2, (points[5][0], points[5][1]), (points[7][0], points[7][1]), lcc, lt)
            cv2.line(path2, (points[7][0], points[7][1]), (points[3][0], points[3][1]), lcc, lt)
            cv2.line(path2, (points[3][0], points[3][1]), (points[6][0], points[6][1]), lcc, lt)
            cv2.line(path2, (points[6][0], points[6][1]), (points[8][0], points[8][1]), lcc, lt)
            
            # caluculate and display score
            if test:
                path_value = 8378
            optimal_score = 54
            percent_score = int(100 * optimal_score / path_value)
            border_color = (1, 1, 1)
            cv2.putText(path2, f'Score: {path_value}    {percent_score}%', (int(8*cqm), int(50*cqm)), cv2.FONT_HERSHEY_TRIPLEX, 2*cqm, border_color, int(2*cqm))
            cv2.putText(path2, f'Score: {path_value}    {percent_score}%', (int(12*cqm), int(50*cqm)), cv2.FONT_HERSHEY_TRIPLEX, 2*cqm, border_color, int(2*cqm))
            cv2.putText(path2, f'Score: {path_value}    {percent_score}%', (int(10*cqm), int(48*cqm)), cv2.FONT_HERSHEY_TRIPLEX, 2*cqm, border_color, int(2*cqm))
            cv2.putText(path2, f'Score: {path_value}    {percent_score}%', (int(10*cqm), int(52*cqm)), cv2.FONT_HERSHEY_TRIPLEX, 2*cqm, border_color, int(2*cqm))
            cv2.putText(path2, f'Score: {path_value}    {percent_score}%', (int(10*cqm), int(50*cqm)), cv2.FONT_HERSHEY_TRIPLEX, 2*cqm, (0, 200, 200), int(2*cqm))
        
            # display instrction text
            cv2.putText(path2, 'press [spacebar to reset]', (int(26*cqm), int(height - 25*cqm)), cv2.FONT_HERSHEY_TRIPLEX, 1*cqm, (1, 1, 1), int(1*cqm))
            cv2.putText(path2, 'press [spacebar to reset]', (int(24*cqm), int(height - 25*cqm)), cv2.FONT_HERSHEY_TRIPLEX, 1*cqm, (1, 1, 1), int(1*cqm))
            cv2.putText(path2, 'press [spacebar to reset]', (int(25*cqm), int(height - 26*cqm)), cv2.FONT_HERSHEY_TRIPLEX, 1*cqm, (1, 1, 1), int(1*cqm))
            cv2.putText(path2, 'press [spacebar to reset]', (int(25*cqm), int(height - 24*cqm)), cv2.FONT_HERSHEY_TRIPLEX, 1*cqm, (1, 1, 1), int(1*cqm))
            cv2.putText(path2, 'press [spacebar to reset]', (int(25*cqm), int(height - 25*cqm)), cv2.FONT_HERSHEY_TRIPLEX, 1*cqm, (150, 150, 150), int(1*cqm))
            
    
    # loop draws the solution paths for every frame
    while complete == True:
        ret, frame = capture.read()
        frame = cv2.flip(frame, 1)
        
        # overlay the path over the webcam feed
        # solution paths
        mask2 = cv2.cvtColor(path2, cv2.COLOR_BGR2GRAY)
        mask2 = cv2.threshold(mask2, 0, 255, cv2.THRESH_BINARY)[1]
        mask2inv = cv2.bitwise_not(mask2)
        pathfg = cv2.bitwise_and(path2, path2, mask=mask2) # extract foreground
        framebg = cv2.bitwise_and(frame, frame, mask=mask2inv) # extract background
        frame = cv2.add(framebg, pathfg) # combine foreground and background
        # original path
        mask2 = cv2.cvtColor(path, cv2.COLOR_BGR2GRAY)
        mask2 = cv2.threshold(mask2, 0, 255, cv2.THRESH_BINARY)[1]
        mask2inv = cv2.bitwise_not(mask2)
        pathfg = cv2.bitwise_and(path, path, mask=mask2) # extract foreground
        framebg = cv2.bitwise_and(frame, frame, mask=mask2inv) # extract background
        frame = cv2.add(framebg, pathfg) # combine foreground and background
        
        cv2.imshow('Supply Chain Maze', frame)
        
        key = cv2.waitKey(10) & 0xFF
        if key == 27:               # 27 is ASCII for [ESC]
            break                   # ends loop when [ESC] is pressed

        key = cv2.waitKey(10) & 0xFF
        if key == 32:               # 32 is ASCII for [spacebar]
            ShortestPath = 'und'
            break                   # ends loop when [spacebar] is pressed

    if key == 27:               # 27 is ASCII for [ESC]
        key = 0
        ShortestPath == 'und'
        break                   # HOLD [ESC] to end program

capture.release()
cv2.destroyAllWindows()  # closes window, only reaches here when [ESC] is pressed